### Indicador de Acessibilidade à Saúde (v5_sau)

O indicador de acessibilidade à saúde foi construído a partir de um escore de capacidade dos estabelecimentos registrados no CNES, alocados em uma malha hexagonal H3 (resolução 9). 

Para mitigar o efeito de fronteira de áreas intraurbanas, aplicou-se uma suavização espacial (decaimento), distribuindo o escore do hexágono de origem para seus vizinhos imediatos com peso reduzido. 

A acessibilidade absoluta (v5_sau_abs) foi calculada pela razão entre a capacidade espacializada e o número de domicílios locais, cujo resultado foi posteriormente winsorizado no 99º percentil para tratamento de outliers e normalizado em uma escala linear de 0 a 1 (v5_sau_norm), onde 1 representa a maior oferta de infraestrutura de saúde perante a demanda local.

In [ ]:
import pandas as pd
import numpy as np
import h3

# =====================================================================
# 1. CARREGAMENTO E PADRONIZAÇÃO DOS DADOS
# =====================================================================
print("Carregando os dados...")

# Carrega a malha H3 do Brasil e padroniza colunas para minúsculo
df_h3 = pd.read_parquet('../data/raw/h3/br_h3_res9.parquet')
df_h3.columns = df_h3.columns.str.lower()

# Carrega os dados do CNES e padroniza colunas para minúsculo
df_cnes = pd.read_csv('../data/raw/cnes/cnes_estabelecimento.csv', sep=';', low_memory=False)
df_cnes.columns = df_cnes.columns.str.lower()


# =====================================================================
# 2. PROCESSAMENTO DO CNES (COORDENADAS E CAPACIDADE)
# =====================================================================
print("Processando dados do CNES...")

# Converte coordenadas para numérico e remove valores nulos
df_cnes['nu_latitude'] = pd.to_numeric(df_cnes['nu_latitude'], errors='coerce')
df_cnes['nu_longitude'] = pd.to_numeric(df_cnes['nu_longitude'], errors='coerce')
df_cnes = df_cnes.dropna(subset=['nu_latitude', 'nu_longitude'])

# Seleciona colunas de serviços para compor a "capacidade" do estabelecimento
servicos = [
    'st_centro_cirurgico', 'st_centro_obstetrico', 'st_centro_neonatal',
    'st_atend_hospitalar', 'st_servico_apoio', 'st_atend_ambulatorial'
]

# Garante que as colunas de serviços sejam numéricas (converte '1.0' string para float)
for col in servicos:
    if col in df_cnes.columns:
        df_cnes[col] = pd.to_numeric(df_cnes[col], errors='coerce').fillna(0)

# Cria o score de capacidade (soma dos serviços + 1 ponto base de existência)
df_cnes['score_capacidade'] = df_cnes[servicos].sum(axis=1) + 1


# =====================================================================
# 3. CONVERSÃO ESPACIAL PARA H3 (RES 9) E AGREGAÇÃO
# =====================================================================
print("Indexando estabelecimentos na malha H3...")

def get_h3_id(lat, lng, res=9):
    try:
        # Compatibilidade com h3-py v3 e v4
        return h3.latlng_to_cell(lat, lng, res) if hasattr(h3, 'latlng_to_cell') else h3.geo_to_h3(lat, lng, res)
    except:
        return None

# Aplica a função para encontrar o hexágono de cada estabelecimento
df_cnes['h3_id'] = np.vectorize(get_h3_id)(df_cnes['nu_latitude'], df_cnes['nu_longitude'])
df_cnes = df_cnes.dropna(subset=['h3_id'])

# Soma a capacidade de todos os estabelecimentos que caíram no mesmo hexágono
df_saude_h3 = df_cnes.groupby('h3_id').agg(
    capacidade_local=('score_capacidade', 'sum')
).reset_index()


# =====================================================================
# 4. DECAIMENTO ESPACIAL (SUAVIZAÇÃO PARA VIZINHOS)
# =====================================================================
print("Aplicando decaimento espacial (k-ring)...")
h3_capacity_dict = {}

for _, row in df_saude_h3.iterrows():
    orig_h3 = row['h3_id']
    cap = row['capacidade_local']
    
    # Obtém vizinhos imediatos (distância 1) - compatível com v3 e v4
    vizinhos = h3.grid_disk(orig_h3, 1) if hasattr(h3, 'grid_disk') else h3.k_ring(orig_h3, 1)
    
    # Hexágono de origem recebe peso 1.0, vizinhos recebem peso 0.5
    for vizinho in vizinhos:
        peso = 1.0 if vizinho == orig_h3 else 0.5
        h3_capacity_dict[vizinho] = h3_capacity_dict.get(vizinho, 0) + (cap * peso)

df_saude_suavizada = pd.DataFrame(
    list(h3_capacity_dict.items()), 
    columns=['h3_id', 'capacidade_suavizada']
)


# =====================================================================
# 5. CÁLCULO DA ACESSIBILIDADE E NORMALIZAÇÃO (WINSORIZAÇÃO)
# =====================================================================
print("Calculando indicadores de acessibilidade...")

# Merge com a malha base do Brasil
df_final = df_h3.merge(df_saude_suavizada, on='h3_id', how='left')

# Preenche com 0 onde não há capacidade de saúde
df_final['capacidade_suavizada'] = df_final['capacidade_suavizada'].fillna(0)

# CÁLCULO ABSOLUTO (v5_sau_abs): Capacidade / Domicílios
# Domicílios = 0 são tratados como NaN temporariamente para evitar divisão por zero
df_final['v5_sau_abs'] = df_final['capacidade_suavizada'] / df_final['qtd_dom'].replace(0, np.nan)
df_final['v5_sau_abs'] = df_final['v5_sau_abs'].fillna(0) # Retorna para 0 onde não tem dom ou não tem saúde

# WINSORIZAÇÃO: Cortando outliers no percentil 99 (evita distorções por valores extremos)
limite_superior = df_final['v5_sau_abs'].quantile(0.99)
df_final['v5_sau_abs_winsorized'] = df_final['v5_sau_abs'].clip(upper=limite_superior)

# NORMALIZAÇÃO MIN-MAX (v5_sau_norm): Transforma a escala para 0 a 1
# 0 = Menor acessibilidade, 1 = Maior acessibilidade
min_val = df_final['v5_sau_abs_winsorized'].min()
max_val = df_final['v5_sau_abs_winsorized'].max()

if max_val > min_val:
    df_final['v5_sau_norm'] = (df_final['v5_sau_abs_winsorized'] - min_val) / (max_val - min_val)
else:
    df_final['v5_sau_norm'] = 0.0

# Limpeza de colunas intermediárias (opcional)
df_final = df_final.drop(columns=['capacidade_suavizada', 'v5_sau_abs_winsorized'])

print("Processo concluído com sucesso!")
print(df_final[['h3_id', 'nm_mun', 'qtd_dom', 'v5_sau_abs', 'v5_sau_norm']].head())

# =====================================================================
# 6. EXPORTAÇÃO
# =====================================================================
df_final.to_parquet('../data/clean/br_h3_saude.parquet', index=False)